Here's the forum that talks about how to get MJD info from the Rubin source catalog:
https://community.lsst.org/t/how-should-one-efficiently-get-the-mjd-for-a-source-catalog/3733/4

In [31]:
# Generic python packages
import pylab as plt
import gc

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst
from datetime import datetime

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

In [32]:
config = 'dp02'
collections = '2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collections)

In [33]:
datasetType = 'calexp'
dataId = {'visit': 192350, 'detector': 175}
calexp = butler.get(datasetType, dataId=dataId)

In [42]:
# This is the time of the observation in MJD:
t1 = calexp.getInfo().getVisitInfo().getDate().get(lsst.daf.base.DateTime.MJD)
print(t1)
# You can also get UTC:
#print(calexp.getInfo().getVisitInfo().getDate().get(lsst.daf.base.DateTime.MJD,lsst.daf.base.DateTime.UTC))


59837.38009422338


### Below, finding the difference between 'now' and the observation:
To do: Is there a way to improve this (take away the 2*365 option)?

In [55]:
# Now get the time of right now and see their difference:

nt = Time.now()
#ut = Time(datetime.utcnow())



if nt.mjd - t1 > 2*365:
    print(f'Obs is older than 2 years {round(nt.mjd - t1,1)} days old')
else:
    print(f'Obs is younger than 2 years {round(nt.mjd - t1,1)} days old')

Obs is younger than 2 years 54.3 days old


### Is there a way to modify the below query to include a time restriction?


In [60]:
# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

import lsst.daf.butler as dafButler
import lsst.geom
import lsst.afw.display as afwDisplay

plt.style.use('tableau-colorblind10')
%matplotlib inline

import warnings
from astropy.units import UnitsWarning
import pandas

#initializing Tap and Butler
pandas.set_option('display.max_rows', 20)
warnings.simplefilter("ignore", category=UnitsWarning)
service = get_tap_service()
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

In [65]:
max_rec=10 # make 100 for full subject set test
use_center_coords = "62, -37"
use_radius = "1.0"

In [ ]:
query = "SELECT TOP " + str(max_rec) + " " + \
        "objectId, coord_ra, coord_dec, detect_isPrimary " + \
        "g_cModelFlux, r_cModelFlux, r_extendedness, r_inputCount " + \
        "FROM dp02_dc2_catalogs.Object " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND detect_isPrimary = 1 " + \
        "AND r_extendedness = 1 " + \
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) < 18.0 " + \
        "ORDER by r_cModelFlux DESC"
results = service.search(query)

assert len(results) == max_rec

In [ ]:
ON fs.ccdVisitId = ccd.ccdVisitId " +